<h1>PREprocessing от Музыкальных Детективов<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Введение" data-toc-modified-id="Введение-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Введение</a></span></li><li><span><a href="#Библиотеки-и-настройки" data-toc-modified-id="Библиотеки-и-настройки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Библиотеки и настройки</a></span><ul class="toc-item"><li><span><a href="#Библиотеки" data-toc-modified-id="Библиотеки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Библиотеки</a></span></li><li><span><a href="#Настройки" data-toc-modified-id="Настройки-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Настройки</a></span></li></ul></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Метрика" data-toc-modified-id="Метрика-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Метрика</a></span></li><li><span><a href="#Модели" data-toc-modified-id="Модели-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

## Введение

**Это вторая из трех тетрадей: EDA, PREprocessing, MODeling.**

**Задача:**

В этом хакатоне вам предлагается разработать решение, которое:

- **Первая задача:** может классифицировать треки по признаку кавер-некавер;
- **Вторая задача:** связывать (группировать) каверы и исходный трек;
- **Третья задача:** находит исходный трек в цепочке каверов.

**Данные:**

Файл covers.json содержит **разметку каверов**, сделанную редакторами сервиса:

- track_id - уникальный идентификатор трека;
- track_remake_type - метка, присвоенная редакторами. Может принимать значения ORIGINAL и COVER;
- original_track_id - уникальный идентификатор исходного трека.

💡 Обратите внимание, что не для всех каверов известны идентификаторы исходных треков!!!

**Метаинформация**

- track_id - уникальный идентификатор трека;
- dttm - первая дата появления информации о треке;
- title - название трека;
- language - язык исполнения;
- isrc - международный уникальный идентификатор трека;
- genres - жанры;
- duration - длительность трека;

**Текст песен**

- track_id - уникальный идентификатор трека;
- lyricId - уникальный идентификатор текста;
- text - текст трека.

## Библиотеки и настройки

### Библиотеки

In [1]:
# !pip install ydata_profiling -U

In [2]:
# ! pip install -U pip setuptools wheel
# ! pip install -U spacy
# ! python -m spacy download en_core_web_sm

In [3]:
! pip install -q optuna

In [4]:
! pip install lightgbm

In [5]:
import pandas as pd 

# # для анализа данных
# from ydata_profiling import ProfileReport

import numpy as np

# для графиков
import matplotlib.pyplot as plt
import seaborn as sns

#для загрузки данных и с сервера и локально
import os
# для скрытия ошибок
import warnings
from IPython.display import display
# для условно рандомных состояний
from numpy.random import RandomState
# регулярные выражения
import re
# #Лемматизатор
# import spacy
# # модель для лемматизации английского языка
# import en_core_web_sm

# разбиение на выборки
from sklearn.model_selection import train_test_split
# перемешивание для upsample
from sklearn.utils import shuffle

from datetime import datetime

# # расчитаем TF-IDF
# from sklearn.feature_extraction.text import TfidfVectorizer
# from nltk.corpus import stopwords
# import nltk

# Логистическая регрессия
from sklearn.linear_model import LogisticRegression
# # Дерево Решений классификатор
# from sklearn.tree import DecisionTreeClassifier
# # Случайный лес классификатор
# from sklearn.ensemble import RandomForestClassifier
# Ridge регрессия
from sklearn.linear_model import RidgeClassifier
# для создания конвеера/ трудопровода
from sklearn.pipeline import Pipeline, make_pipeline
# для маштабирования признаков
from sklearn.preprocessing import StandardScaler
# заполнение пропусков
from sklearn.impute import SimpleImputer
# векторизация текста
from sklearn.feature_extraction.text import TfidfVectorizer
# обработка категориальных признаков
from sklearn.preprocessing import OrdinalEncoder
# обработка категориальных признаков
from sklearn.preprocessing import OneHotEncoder
# для раздельной предобработки количествнных и категорийных признаков
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier

# F1 метрика
from sklearn.metrics import f1_score

# # для подбора параметров моделей
# from sklearn.model_selection import GridSearchCV

# оптимизация параметров моделей
import optuna

import lightgbm as lgb

### Настройки

In [6]:
# для скрытия ошибок
warnings.filterwarnings("ignore")

# для увеличения окна вывода 
pd.options.display.max_rows = 100
# максимальная вместимость ячейки для отображения
pd.set_option('display.max_colwidth', None)
# четыре знака после запятой
pd.set_option('display.float_format', '{:.4f}'.format)

# для задания размера графиков по умолчанию
plt.rcParams["figure.figsize"] = (7, 5)

# для псевдорандомных значений
np.random.seed(seed=54321)
np.random.RandomState(seed=54321)
RS=54321

# # загрузим английские стоп слова
# nltk.download('stopwords')
# stop_words = set(stopwords.words('english'))

## Загрузка данных

In [7]:
path_features_train = 'data/preprocessing/features_train.csv'

features_train = pd.read_csv(path_features_train)
features_train['dttm'] = pd.to_datetime(features_train['dttm'], unit='ns', origin='unix').astype('int')
features_train.info()
features_train.head()



<class 'pandas.core.frame.DataFrame'>
Index: 41090 entries, 9748 to 24495
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title_lemm       41019 non-null  object 
 1   language         12351 non-null  object 
 2   title_language   41043 non-null  object 
 3   isrc_country     40905 non-null  object 
 4   isrc_prod_centr  40905 non-null  object 
 5   genres_big_cat   41090 non-null  object 
 6   dttm             41090 non-null  int64  
 7   duration         41090 non-null  int64  
 8   title_len        41090 non-null  int64  
 9   isrc_year        40905 non-null  float64
 10  isrc_reg_number  40905 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 3.8+ MB


,title_lemm,language,title_language,isrc_country,isrc_prod_centr,genres_big_cat,dttm,duration,title_len,isrc_year,isrc_reg_number
9748,smell like teen spirit,EN,EN,GB,PW4,ROCK,1486692362000000000,307880,23,2015.0000,20845.0000
57013,illness of you,NaN,ES,QZ,5AB,POP,1643334417000000000,216190,16,2022.0000,42806.0000
54292,traitor,NaN,EN,QZ,PJ3,ALTERNATIVE,1639602429000000000,229780,7,2021.0000,29867.0000
2316,in the end,NaN,EN,US,A56,METAL,1320928892000000000,217120,10,2008.0000,98778.0000
25572,we will be,NaN,ES,QZ,NJW,FOLK,1603727794000000000,139700,7,2020.0000,24077.0000


In [8]:
path_features_train_upsamled = 'data/preprocessing/features_train_upsamled.csv'

features_train_upsamled = pd.read_csv(path_features_train_upsamled)
features_train_upsamled['dttm'] = pd.to_datetime(
    features_train_upsamled['dttm'], unit='ns', origin='unix').astype('int')
features_train_upsamled.info()
features_train_upsamled.head()

<class 'pandas.core.frame.DataFrame'>
Index: 77140 entries, 68233 to 67280
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title_lemm       76901 non-null  object 
 1   language         32371 non-null  object 
 2   title_language   77037 non-null  object 
 3   isrc_country     76171 non-null  object 
 4   isrc_prod_centr  76171 non-null  object 
 5   genres_big_cat   77140 non-null  object 
 6   dttm             77140 non-null  int64  
 7   duration         77140 non-null  int64  
 8   title_len        77140 non-null  int64  
 9   isrc_year        76171 non-null  float64
 10  isrc_reg_number  76171 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 7.1+ MB


,title_lemm,language,title_language,isrc_country,isrc_prod_centr,genres_big_cat,dttm,duration,title_len,isrc_year,isrc_reg_number
68233,I m so lucky,NaN,RU,RU,B42,POP,1687467600000000000,204930,15,2023.0000,1729.0000
68278,forget,RU,RU,RU,AC4,POP,1689282000000000000,202100,6,2023.0000,201.0000
67747,heart in half,RU,RU,FR,10S,RAP,1674162000000000000,203720,16,2023.0000,73214.0000
67559,tonight,NaN,EN,RU,AGT,OTHER,1672347600000000000,128210,7,2022.0000,9328.0000
49099,coffee with raindrop,RU,RU,RU,ACR,POP,1634763600000000000,151380,20,2021.0000,343.0000


In [9]:
path_features_valid = 'data/preprocessing/features_valid.csv'

features_valid = pd.read_csv(path_features_valid)
features_valid['dttm'] = pd.to_datetime(features_valid['dttm'], 
                                        unit='ns', origin='unix').astype('int')
features_valid.info()
features_valid.head()

<class 'pandas.core.frame.DataFrame'>
Index: 13697 entries, 32921 to 2690
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title_lemm       13673 non-null  object 
 1   language         4166 non-null   object 
 2   title_language   13683 non-null  object 
 3   isrc_country     13644 non-null  object 
 4   isrc_prod_centr  13644 non-null  object 
 5   genres_big_cat   13697 non-null  object 
 6   dttm             13697 non-null  int64  
 7   duration         13697 non-null  int64  
 8   title_len        13697 non-null  int64  
 9   isrc_year        13644 non-null  float64
 10  isrc_reg_number  13644 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 1.3+ MB


,title_lemm,language,title_language,isrc_country,isrc_prod_centr,genres_big_cat,dttm,duration,title_len,isrc_year,isrc_reg_number
32921,steal kiss,NaN,PT,QZ,GLS,FOLK,1617267337000000000,152490,13,2021.0000,46491.0000
17112,swan song,EN,EN,FR,59R,OTHER,1562576783000000000,182710,9,2019.0000,58134.0000
50472,all about you,NaN,ES,QZ,NJY,ALTERNATIVE,1635371163000000000,202000,10,2021.0000,8868.0000
41933,edge of the earth,EN,EN,QZ,S63,ROCK,1628629200000000000,294000,17,2021.0000,78917.0000
38544,act,NaN,PT,QZ,HZ5,FOLK,1624541502000000000,176930,7,2021.0000,61294.0000


In [10]:
path_features_test = 'data/preprocessing/features_test.csv'

features_test = pd.read_csv(path_features_test)
features_test['dttm'] = pd.to_datetime(features_test['dttm'], 
                                       unit='ns', origin='unix').astype('int')
features_test.info()
features_test.head()

<class 'pandas.core.frame.DataFrame'>
Index: 13697 entries, 35892 to 27147
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title_lemm       13677 non-null  object 
 1   language         4144 non-null   object 
 2   title_language   13684 non-null  object 
 3   isrc_country     13627 non-null  object 
 4   isrc_prod_centr  13627 non-null  object 
 5   genres_big_cat   13697 non-null  object 
 6   dttm             13697 non-null  int64  
 7   duration         13697 non-null  int64  
 8   title_len        13697 non-null  int64  
 9   isrc_year        13627 non-null  float64
 10  isrc_reg_number  13627 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 1.3+ MB


,title_lemm,language,title_language,isrc_country,isrc_prod_centr,genres_big_cat,dttm,duration,title_len,isrc_year,isrc_reg_number
35892,hunt rare devil,NaN,EN,QZ,GWX,ROCK,1621439750000000000,169160,20,2021.0000,91089.0000
35440,k chler concert violin op mov,NaN,EN,QZ,HZ4,OTHER,1620676022000000000,242100,37,2021.0000,39448.0000
31958,the man who sell the world,EN,EN,GB,EQT,RNB,1618434000000000000,208480,26,2021.0000,20.0000
16322,after midnight,EN,EN,IT,AD8,OTHER,1557090000000000000,182000,14,2019.0000,37.0000
46927,another dimension,NaN,ES,AR,E27,RNB,1632173582000000000,216720,14,2021.0000,18.0000


In [11]:
path_target_train = 'data/preprocessing/target_train.csv'

target_train = pd.read_csv(path_target_train)
display(target_train.describe())
target_train.head()

,track_remake_type
count,77140.0000
mean,0.5007
std,0.5000
min,0.0000
25%,0.0000
50%,1.0000
75%,1.0000
max,1.0000


,track_remake_type
68233,1
68278,1
67747,1
67559,1
49099,1


In [12]:
path_target_train_upsampled = 'data/preprocessing/target_train_upsampled.csv'

target_train_upsampled = pd.read_csv(path_target_train_upsampled)
display(target_train_upsampled.describe())
target_train.head()

,track_remake_type
count,77140.0000
mean,0.5007
std,0.5000
min,0.0000
25%,0.0000
50%,1.0000
75%,1.0000
max,1.0000


,track_remake_type
68233,1
68278,1
67747,1
67559,1
49099,1


In [13]:
path_target_valid = 'data/preprocessing/target_valid.csv'

target_valid = pd.read_csv(path_target_valid)
display(target_valid.describe())
target_valid.head()

,track_remake_type
count,13697.0000
mean,0.0627
std,0.2425
min,0.0000
25%,0.0000
50%,0.0000
75%,0.0000
max,1.0000


,track_remake_type
32921,0
17112,0
50472,0
41933,0
38544,0


In [14]:
path_target_test = 'data/preprocessing/target_test.csv'

target_test = pd.read_csv(path_target_test)
display(target_test.describe())
target_test.head()

,track_remake_type
count,13697.0000
mean,0.0626
std,0.2423
min,0.0000
25%,0.0000
50%,0.0000
75%,0.0000
max,1.0000


,track_remake_type
35892,0
35440,0
31958,0
16322,1
46927,0


## Метрика

## Модели

In [15]:
features_train_upsamled.columns

Index(['title_lemm', 'language', 'title_language', 'isrc_country',
       'isrc_prod_centr', 'genres_big_cat', 'dttm', 'duration', 'title_len',
       'isrc_year', 'isrc_reg_number'],
      dtype='object')

In [16]:
# запишем обучающие признаки в соответствующие списки
features_list_text = ['title_lemm']
features_list_cat = ['language', 'title_language', 'isrc_country',
                     'isrc_prod_centr', 'genres_big_cat']

features_list_num = ['dttm', 'duration', 'title_len', 'isrc_year','isrc_reg_number']

In [17]:
category_features_index = []

for col in features_list_cat:
    category_features_index.append(features_train_upsamled.columns.get_loc(col))
    
category_features_index

[1, 2, 3, 4, 5]

In [18]:
# задаем шаги в Pipeline

# обработка численных признаков для базового алгоритма Линейная регрессия
# заполнение 0 и стандартизация с маштабированием
num_steps_linear = [('imputer',  SimpleImputer(missing_values=np.nan
                                        , strategy='constant'
                                        , fill_value=0
                                        , add_indicator=True
                                        )
                    )
                    , ('scaler', StandardScaler()
                       )
                     ]
num_preprocessor_linear = Pipeline(num_steps_linear)
                     
# обработка численных признаков для базового алгоритма Дерево решений
# заполнение аномальными значениями
num_steps_tree = [('imputer',  SimpleImputer(missing_values=np.nan
                                            , strategy='constant'
                                            , fill_value=-1000
                                            )
                  )
                 ]
num_preprocessor_tree = Pipeline(num_steps_tree)                     

# обработка категориальных признаков для базового алгоритма Линейная регрессия
category_steps_linear = [('imputer', SimpleImputer(missing_values=np.nan, 
                                                   strategy='constant', 
                                                   fill_value='unknown'))
                         , ('encoder', OrdinalEncoder(handle_unknown = 'ignore'))
                         , ('scaler', StandardScaler())
                        ]


category_preprocessor_linear = Pipeline(category_steps_linear)

# обработка категориальных признаков для базового алгоритма Дерово решений
category_steps_tree = [('imputer', SimpleImputer(missing_values=np.nan, 
                                                 strategy='constant', 
                                                 fill_value='unknown'))
                       ,('encoder', OrdinalEncoder(handle_unknown = 'ignore', 
#                                                    dtype=np.uint32
                                                  )
                        )
                      ]
category_preprocessor_tree = Pipeline(category_steps_tree)

# обработка текстовых признаков для линейных алгоритмов
text_step_linear = [ ('vect', TfidfVectorizer()),
#                    ('scaler', StandardScaler()) ?
                   ]
text_preprocessor_linear = Pipeline(text_step_linear)

# обработка текстовых признаков для базового алгоритма Дерово решений
text_step_tree = [ ('vect', TfidfVectorizer())]

text_preprocessor_tree = Pipeline(text_step_tree)


# собираем все вместе для базового алгоритма Линейная регрессия
preprocessor_linear = ColumnTransformer(transformers=[('num', num_preprocessor_linear, features_list_num)
                                                      , ('cat', category_preprocessor_linear, features_list_cat)
                                                      , ('text', text_preprocessor_linear, features_list_text)
                                                     ]
                                        , remainder='passthrough'
                                       )

# собираем все вместе для базового алгоритма Дерево Решений
preprocessor_tree = ColumnTransformer(transformers=[('num', num_preprocessor_tree, features_list_num),
                                                    ('cat', category_preprocessor_tree, features_list_cat),
                                                    ('text', text_preprocessor_tree, features_list_text)
                                                   ]
                                      , remainder='passthrough'
                                     )


In [19]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 1000, step=10)
    max_depth = trial.suggest_int('max_depth', 4, 50)
    num_leaves = trial.suggest_int('num_leaves', 10, 150, step=10)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.9, step=None, log=True)
    
    pipeline_lgbm_classifier = Pipeline([
        ('preprocessor', preprocessor_tree),
        ('classifier', lgb.LGBMClassifier(boosting_type='gbdt',
                                          num_leaves=num_leaves,
                                          max_depth=max_depth,
                                          learning_rate=learning_rate,
                                          objective= 'binary', 
                                          n_estimators=n_estimators,
                                          random_state=RS,
                                          n_jobs=-1
                                         )
         )
        ]
        )
#     pipeline_lgbm_classifier.fit(features_train
# #                                  .drop(columns='title_lemm')
#                                  ,
#                                  target_train,
#                                  classifier__categorical_feature=category_features_index
# #                                  classifier__categorical_feature=features_list_cat
#                                  )

#     f_one = f1_score(target_train,
#                      pipeline_lgbm_classifier.predict(features_train
# #                                                       .drop(columns='title_lemm')
#                                                      )
#                     )
    
    pipeline_lgbm_classifier.fit(features_train_upsamled
#                                  .drop(columns='title_lemm')
                                 ,
                                 target_train_upsampled,
#                                  classifier__categorical_feature=category_features_index
                                 classifier__categorical_feature=features_list_cat
                                 )

    f_one = f1_score(target_train_upsampled,
                     pipeline_lgbm_classifier.predict(features_train_upsamled
#                                                       .drop(columns='title_lemm')
                                                     )
                    )
#     f_one = f1_score(target_valid,
#                      pipeline_lgbm_classifier.predict(features_valid
# #                                                       .drop(columns='title_lemm')
#                                                      )
#                     )
    return f_one


study = optuna.create_study(direction='maximize'
                            , sampler=optuna.samplers.RandomSampler(seed=RS)
                            , pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
                           )
study.optimize(objective, n_trials=100, timeout=600)
study.best_params

[I 2023-11-01 16:29:06,073] A new study created in memory with name: no-name-d5657e8e-2366-4349-9880-85aee9b2aa7d
[W 2023-11-01 16:29:06,190] Trial 0 failed with parameters: {'n_estimators': 920, 'max_depth': 33, 'num_leaves': 120, 'learning_rate': 0.01860752093274472} because of the following error: ValueError('all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 77140 and the array at index 2 has size 1').
Traceback (most recent call last):
  File "/home/tomasha1980/anaconda3/envs/ds_practicum_env/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_7705/3594213034.py", line 35, in objective
    pipeline_lgbm_classifier.fit(features_train_upsamled
  File "/home/tomasha1980/anaconda3/envs/ds_practicum_env/lib/python3.9/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  F

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 77140 and the array at index 2 has size 1

In [ ]:
# запишем наш pipeline_lgbm_classifier_best с лучшими параметрами
pipeline_lgbm_classifier_best = Pipeline([
        ('preprocessor', preprocessor_tree),
        ('classifier', lgb.LGBMClassifier(boosting_type='gbdt',
                                          num_leaves=study.best_params['num_leaves'],
                                          max_depth=study.best_params['max_depth'],
                                          learning_rate=study.best_params['learning_rate'],
                                          objective= 'binary', 
                                          n_estimators=study.best_params['n_estimators'],
                                          random_state=RS,
                                          n_jobs=-1,)
         )
        ]
        )
pipeline_lgbm_classifier

In [ ]:
# обучим на сбалансированной выборке
pipeline_lgbm_classifier_best.fit(features_train_upsamled, target_train_upsampled)

In [ ]:
pipeline_lgbm_classifier_best.steps[1]

In [ ]:
# проверим на проверочно выборке
f_one = f1_score(target_valid,
                     pipeline_lgbm_classifier_best.predict(features_valid
#                                                       .drop(columns='title_lemm')
                                                     )
                )
f_one                  

In [ ]:
# проверим на тестовой выборке выборке
f_one = f1_score(target_test,
                 pipeline_lgbm_classifier_best.predict(features_test
#                                                       .drop(columns='title_lemm')
                                                     )
                )
f_one                  

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [21]:
dummy_model = DummyClassifier(strategy="most_frequent")
dummy_model.fit(features_train_upsamled, target_train_upsampled)

f_one = f1_score(target_valid, dummy_model.predict(features_valid)
f_one


SyntaxError: invalid syntax (1564007043.py, line 5)

## Вывод